# Data loading

In [ ]:
import numpy as np
from IPython.display import display, Image

from platform import python_version
print(python_version())

In [ ]:
# data is in float64 format: 0.0-0.000255
def load_data(type, region):
    # `raw_validation_set.npy` is multiple tries non-averaged `validation_set.npy`
    return (
        (np.load(f'./Data/region{region}/{type}_inputs.npy')*1_000_000).astype(np.uint8),
        np.load(f'./Data/region{region}/{type}_set.npy')
    )
    
(input_tr, output_tr) = load_data('training', 2) 
(val_input, val_output) = load_data('validation', 2)

# Data exploration

In [ ]:
display(input_tr.shape, output_tr.shape)
display(val_input.shape, val_output.shape)

In [ ]:
display(input_tr.shape)
display(input_tr[0])
display(input_tr.dtype)
display(min(input_tr[0]), max(input_tr[0]), np.mean(input_tr[0]), np.std(input_tr[0]))
display(min(input_tr[1]), max(input_tr[1]), np.mean(input_tr[1]), np.std(input_tr[1]))

display(output_tr.shape)
display(output_tr[0])
display(output_tr.dtype)
display(min(output_tr[0]), max(output_tr[0]), np.mean(output_tr[0]), np.std(output_tr[0]))


In [ ]:
from PIL import Image

def reshape_single_as_picture(input, size=31):
    return np.reshape(input, (size, size))

def as_single_picture(input, size=31):
    return Image.fromarray(reshape_single_as_picture(input, size), 'L')

In [ ]:
display(as_single_picture(input_tr[0]))
display(as_single_picture(input_tr[2]))
display(as_single_picture(input_tr[700]))
display(as_single_picture(input_tr[-1]))
display(as_single_picture(input_tr[-2]))

display(input_tr[0]*1_000_000)

# Data trasnform

In [ ]:
def downsample_input(dta):
    def process_single(pic):
        resized_pic = as_single_picture(pic).resize(size=(15, 15), resample=Image.BICUBIC)
        return np.array(resized_pic).reshape(15*15)
    return np.array([process_single(pic) for pic in dta])

def normalize_input(dta):
    return (dta/np.std(dta))

In [ ]:
input_tr_processed = downsample_input(input_tr)
display(as_single_picture(input_tr_processed[0], 15))
display(as_single_picture(input_tr_processed[2], 15))
display(as_single_picture(input_tr_processed[-1], 15))
display(as_single_picture(input_tr_processed[-2], 15))

display(input_tr_processed[0], min(input_tr_processed[0]), max(input_tr_processed[0]))

# Network parameters

In [ ]:
import NDN3.NDNutils as NDNutils
import NDN3.NDN as NDN

from datetime import datetime

In [ ]:
def get_hsm_params(input, output, hls=40):
    _, output_shape = output.shape
    _, input_shape = input.shape
    display(f"in: {input_shape} out: {output_shape}")

    d2x = 0.0005
    l1 = 0.000001

    hsm_params = NDNutils.ffnetwork_params(
        input_dims=[1, input_shape], 
        layer_sizes=[hls, 2*hls, output_shape],
        ei_layers=[0, hls // 2],
        normalization=[0], 
        layer_types=['normal','normal','normal'], 
        reg_list={
            'd2x':[d2x,None,None],
            'l1':[l1,None,None],
            'max':[None,None,100]})
    hsm_params['weights_initializers']=['normal','normal','normal']
    hsm_params['normalize_weights']=[1,0,0]
    
    return hsm_params

# Network training

In [ ]:
def train_network(train_input, train_output, larg, opt_params, hsm_params, test_input = None, test_output = None):
    time_str = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    display(time_str)
    
    train_len, _ = train_input.shape
    if test_input is not None:
        input = np.concatenate([train_input, test_input], axis=0)
        output = np.concatenate([train_output, test_output], axis=0)
        test_len, _ = test_input.shape
    else:
        input = train_input
        output = train_output
        opt_params['early_stop'] = 0 # If we don't have test data -> shoudn't be early stopping (could early stop on train)
        test_len = 0  
        
    train_indxs = np.array(range(train_len))
    test_indxs = np.array(range(train_len, train_len + test_len)) if test_len > 0 else None
    
    hsm = NDN.NDN(hsm_params, noise_dist='poisson')
    hsm.train(
        input_data=input, 
        output_data=output, 
        train_indxs=train_indxs, 
        test_indxs=test_indxs, 
        learning_alg=larg, 
        opt_params=opt_params, 
        output_dir=f"logs/{time_str}"
    )
    
    return hsm

# Results evaluation

In [ ]:
def get_correlation(a, b):
    import scipy.stats
    assert a.shape == b.shape
    c = [scipy.stats.pearsonr(a[:,i], b[:,i])[0] for i in range(a.shape[1])]
    return np.array(c)

In [ ]:
def show_some_first_layers(hsm):
    import matplotlib.pyplot as plt  # plotting

    input_size = [15, 15]
    nrows, ncols = 5, 8

    fig, _ = plt.subplots(nrows=nrows, ncols=ncols)
    fig.set_size_inches(12, 6)
    for neuron_i in range(nrows * ncols):
        plt.subplot(nrows, ncols, neuron_i+1)
        w = hsm.networks[0].layers[0].weights[:,neuron_i]
        plt.imshow(np.reshape(w, input_size), cmap='Greys', interpolation='none', vmin=-max(abs(w)), vmax=max(abs(w)))
    plt.show()
    

In [ ]:
def print_stat_matrix(string, stat):
    display(string + ": " + str(np.mean(stat)))
    display(stat)
    
def evaluate_all(hsm, input, golden):
    dta_len, _ = input.shape
    
    eval = hsm.eval_models(input_data=input, output_data=golden, data_indxs=np.array(range(dta_len)), nulladjusted=True) 
    print_stat_matrix("Null adj eval ", eval)
    
    result = hsm.generate_prediction(input)
    std_result, std_golden = np.std(result, axis=0), np.std(golden, axis=0)
    
    print_stat_matrix("STD result", std_result)
    print_stat_matrix("STD golden", std_golden)
    
    corr = get_correlation(result, golden)
    corr[np.isnan(corr)] = 0
    print_stat_matrix("Corr", corr)
    
    return result

In [ ]:
def print_stat_scalar(string, stat):
    display(string + ": " + str(stat))

def evaluate_output_neuron(result, golden, i):
    corr = get_correlation(result, golden)
    corr[np.isnan(corr)] = 0
    print_stat_scalar("Corr", corr[i])
    
    print_stat_scalar("STD result", np.std(result, axis=0)[i])
    print_stat_scalar("STD golden", np.std(golden, axis=0)[i])
    
    print_stat_scalar("Mean result", np.mean(result, axis=0)[i])
    print_stat_scalar("Mean golden", np.mean(golden, axis=0)[i])

    display("Example result", result[:15, i])
    display("Example golden", golden[:15, i])
    
def evaluate_best_corr_neuron(result, golden):
    corr = get_correlation(result, golden)
    corr[np.isnan(corr)] = 0

    max_corr_i = np.nanargmax(corr)
    print_stat_scalar("Argmax i", max_corr_i)
    
    evaluate_output_neuron(result, golden, max_corr_i)

# Experiments

In [ ]:
(input_tr, output_tr) = load_data('training', 2) 
input_tr_processed = downsample_input(input_tr)

(input_val, output_val) = load_data('validation', 2)
input_val_processed = downsample_input(input_val)

hsm_params = get_hsm_params(input_tr_processed, output_tr)
hsm = train_network(
    input_tr_processed, output_tr,
    'lbfgs', 
    {'batch_size': 2, 'use_gpu': False, 'epochs_summary': 10, 'epochs_training': 300, 'maxiter': 2000, 'display': True, 'ftol': 2.220446049250313e-11},
    hsm_params,
    input_val_processed, output_val
)


res = evaluate_all(hsm, input_val_processed, output_val)
show_some_first_layers(hsm)
evaluate_best_corr_neuron(res, output_val)

In [ ]:
(input_tr, output_tr) = load_data('training', 2) 
input_tr_processed = downsample_input(input_tr)

(input_val, output_val) = load_data('validation', 2)
input_val_processed = downsample_input(input_val)

hsm_params = get_hsm_params(input_tr_processed, output_tr)
hsm = train_network(
    input_tr_processed, output_tr,
    'adam', 
    {'batch_size': 8, 'use_gpu': False, 'epochs_summary': 10, 'epochs_training': 1000, 'learning_rate': 0.5e-4},
    hsm_params,
    input_val_processed, output_val
)


res = evaluate_all(hsm, input_val_processed, output_val)
show_some_first_layers(hsm)
evaluate_best_corr_neuron(res, output_val)